In [1]:
%reset -f

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

data_path = '/Users/ayemyatwinshwe/repos/data/trading_data/'

In [3]:
df_tokens = pd.read_parquet(data_path + 'earning_call_tokens')
df_tokens.head()

,tokens
0,"[operator, good, day, ladies, gentlemen, welco..."
1,"[operator, good, day, ladies, gentlemen, welco..."
2,"[operator, good, afternoon, welcome, agilent, ..."
3,"[operator, good, afternoon, welcome, agilent, ..."
4,"[operator, good, afternoon, welcome, agilent, ..."


In [4]:
data_tokens = df_tokens['tokens'].to_list()
del df_tokens

In [5]:
%%time
# bigrams and trigrams
bigrams_phrases = Phrases(data_tokens, 
                          min_count= 25, 
                          threshold = 1)
            
trigrams_phrases = Phrases(bigrams_phrases[data_tokens], 
                           min_count= 25, 
                           threshold = 1)

bigram = Phraser(bigrams_phrases)
trigram = Phraser(trigrams_phrases)

data_bigrams = [bigram[doc] for doc in data_tokens]
data_trigrams = [trigram[bigram[doc]] for doc in data_bigrams]

CPU times: user 9min 45s, sys: 40.6 s, total: 10min 26s
Wall time: 10min 54s


%%time
model = Word2Vec(sentences=data_trigrams, 
                 sg=1,
                 hs=0,
                 vector_size=300, 
                 window=5, 
                 min_count=50,
                 negative=15,
                 workers=4,
                 alpha=0.05,
                 min_alpha=0.0001)

model.save("word2vec.model")

In [ ]:
model = Word2Vec.load("word2vec.model")
word_vectors = model.wv
word_vectors.save("vectors.kv")
del model
del word_vectors

In [6]:
wv = KeyedVectors.load('vectors.kv')

In [7]:
%%time
word_vecs = []

for i in np.arange(len(data_trigrams)):
    vecs = []
    for x in data_trigrams[i]:
        try:
            vecs.append(wv[x])
        except KeyError:
            pass
    word_vecs.append(vecs)

CPU times: user 1min 16s, sys: 24.1 s, total: 1min 40s
Wall time: 1min 47s


In [8]:
df_word_vecs = pd.DataFrame()
df_word_vecs['vectors'] = word_vecs
df_word_vecs

,vectors
0,"[[1.0666107, -0.28468454, -0.08754885, -0.4464..."
1,"[[1.0666107, -0.28468454, -0.08754885, -0.4464..."
2,"[[0.6297637, -0.32820508, 0.02398203, -0.13134..."
3,"[[0.6297637, -0.32820508, 0.02398203, -0.13134..."
4,"[[0.6297637, -0.32820508, 0.02398203, -0.13134..."
...,...
26232,"[[0.6297637, -0.32820508, 0.02398203, -0.13134..."
26233,"[[0.7034699, -0.28815025, -0.30461752, 0.17155..."
26234,"[[0.7034699, -0.28815025, -0.30461752, 0.17155..."
26235,"[[0.6297637, -0.32820508, 0.02398203, -0.13134..."
